In [207]:
import numpy as np


In [11]:
a = np.arange(24).reshape(4,6)
u,s,v = np.linalg.svd(a)
u.shape,v.shape,s.shape

((4, 4), (6, 6), (4,))

In [12]:
s

array([6.56235089e+01, 4.18987869e+00, 2.31759128e-15, 1.33649655e-15])

In [13]:
def retore(sigma,u,v,k):
    u1 = u[:,:k]
    v1 = v[:k,:]
    s1 = np.eye(k)
    for i in range(k):
        s1[i,i] = sigma[i]
    return np.dot(np.dot(u1,s1),v1)

In [15]:
a

array([[ 0,  1,  2,  3,  4,  5],
       [ 6,  7,  8,  9, 10, 11],
       [12, 13, 14, 15, 16, 17],
       [18, 19, 20, 21, 22, 23]])

In [14]:
retore(s,u,v,2)

array([[-1.89149162e-15,  1.00000000e+00,  2.00000000e+00,
         3.00000000e+00,  4.00000000e+00,  5.00000000e+00],
       [ 6.00000000e+00,  7.00000000e+00,  8.00000000e+00,
         9.00000000e+00,  1.00000000e+01,  1.10000000e+01],
       [ 1.20000000e+01,  1.30000000e+01,  1.40000000e+01,
         1.50000000e+01,  1.60000000e+01,  1.70000000e+01],
       [ 1.80000000e+01,  1.90000000e+01,  2.00000000e+01,
         2.10000000e+01,  2.20000000e+01,  2.30000000e+01]])

In [25]:
# 欧氏距离计算相似度(转换到0-1区间,越相似，值越大)
def ecludSim(inA,inB):
    return 1.0/(1.0+np.linalg.norm(inA-inB))

# 皮尔逊相关系数计算相似度（对数值本身不关心，范围-1-1，转换到0-1）
def pearsSim(inA,inB):
    if len(inA)<3 : 
        return 1.0
    return 0.5+0.5*np.corrcoef(inA,inB,rowvar=0)[0][1]
# 余弦相似度计算相似度（范围-1-1，转换到0-1）
def cosSim(inA,inB):
    num = float(inA.T*inB)
    denom = np.linalg.norm(inA)*np.linalg.norm(inB)
    return 0.5+0.5*(num/denom)

In [21]:
myMat = np.mat(loadExData())
ecludSim(myMat[:,0],myMat[:,4])

0.13367660240019172

In [22]:
ecludSim(myMat[:,0],myMat[:,0])

1.0

In [23]:
pearsSim(myMat[:,0],myMat[:,4])

0.23768619407595815

In [24]:
pearsSim(myMat[:,0],myMat[:,0])

1.0

In [25]:
cosSim(myMat[:,0],myMat[:,4])

0.5472455591261534

In [26]:
cosSim(myMat[:,0],myMat[:,0])

0.9999999999999999

In [194]:
# 数据矩阵，用户编号，和相似度的计算方法，未评分下标
def standEst(dataMat,user,simMeas,item):
    # 获取总列数
    n = dataMat.shape[1]
    simTotal,ratSimTotal = 0.0,0.0
    for j in range(n):
        userRaing = dataMat[user,j]
        if userRaing == 0:
            continue
        # 获取两物品评分相同的下标
        overLap = np.nonzero(np.logical_and(dataMat[:,item].A>0,dataMat[:,j].A>0))[0]
        if len(overLap) == 0:
            similarity = 0
        else:
            # 将两物品进行相似度计算
            similarity = simMeas(dataMat[overLap,item],dataMat[overLap,j])
        # 相似度累加
        simTotal += similarity
        # 预测分值（相似度*参照分值）
        ratSimTotal+=similarity*userRaing
    if simTotal ==0:
        return 0
    else:
        return ratSimTotal/simTotal

In [195]:
# 数据矩阵，用户编号，相似度计算方式，预测分值方式
def recommend(dataMat,user,N=3,simMeas=cosSim,estMethod=standEst):
    # 返回值为0的下标(该用户未评分的物品)
    unratedItems = np.nonzero(dataMat[user,:].A==0)[1]
    if len(unratedItems) == 0:
        return "已全部评价"
    itemScores=[]
    for item in unratedItems:
        estimatedscore = estMethod(dataMat,user,simMeas,item)
        itemScores.append((item,estimatedscore))
    return sorted(itemScores,key=lambda jj:jj[1],reverse=True)[:N]

In [196]:
def loadExData():
    return [[4, 4, 0, 2, 2],
             [4, 0, 0, 3, 3],
             [4, 0, 0, 1, 1],
             [1, 1, 1, 2, 0],
             [2, 2, 2, 0, 0],
             [1, 1, 1, 0, 0],
             [5, 5, 5, 0, 0]]

In [197]:
myMat = np.mat(loadExData())
recommend(myMat,2)

[(2, 2.5), (1, 2.0243290220056256)]

In [198]:
def loadExData2():
    return[[2, 0, 0, 4, 4, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5],
           [0, 0, 0, 0, 0, 0, 0, 1, 0, 4, 0],
           [3, 3, 4, 0, 3, 0, 0, 2, 2, 0, 0],
           [5, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 5, 0, 0, 5, 0],
           [4, 0, 4, 0, 0, 0, 0, 0, 0, 0, 5],
           [0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 4],
           [0, 0, 0, 0, 0, 0, 5, 0, 0, 5, 0],
           [0, 0, 0, 3, 0, 0, 0, 0, 4, 5, 0],
           [1, 1, 2, 1, 1, 2, 1, 0, 4, 5, 0]]

In [199]:
U,Sigma,VT = np.linalg.svd(np.mat(loadExData2()))
Sigma

array([1.34342819e+01, 1.18190832e+01, 8.20176076e+00, 6.86912480e+00,
       5.29063022e+00, 3.91213561e+00, 2.94562509e+00, 2.35486137e+00,
       2.08702082e+00, 7.08715931e-01, 1.90990329e-16])

In [200]:
sig2 = Sigma**2

In [201]:
np.sum(sig2)*0.9

447.3

In [202]:
np.sum(sig2[:2])

320.17065834028847

In [203]:
np.sum(sig2[:3])

387.4395378556578

In [204]:
np.sum(sig2[:5])

462.61518152879415

In [205]:
def svdEst(dataMat, user, simMeas, item):
    n = dataMat.shape[1]
    simTotal,ratSimTotal = 0.0,0.0
    U,Sigma,VT = np.linalg.svd(dataMat)
    Sig4 = np.mat(np.eye(4)*Sigma[:4]) 
    # 将物品转换到低维空间中
    xformedItems = dataMat.T * U[:,:4] * Sig4.I  
    for j in range(n):
        userRating = dataMat[user,j]
        if userRating == 0 or j==item: 
            continue
        similarity = simMeas(xformedItems[item,:].T,xformedItems[j,:].T)
        simTotal += similarity
        ratSimTotal += similarity * userRating
    if simTotal == 0: 
        return 0
    else: 
        return ratSimTotal/simTotal

In [206]:
recommend(myMat,1,estMethod=svdEst)

[(2, 3.417756918659238), (1, 3.330717154558564)]